# MQTT Plot

The need for plotting sensor data arises frequently - in fact we have such a need immediately to develop and test the robot in the next section.

`MQTT Plot` is a simple Python app that plots  data sent by MQTT (e.g. directly from a microcontroller) and renders it in a dynamic plot. Update the MQTT broker address in `mqtt_plot/config.py` and other parameters as appropriate to match your setup.

The following messages are recognized (`MQTT_TOPIC_ROOT` is defined in mqtt_plot/config.py):

## Data

* Initialize plot:
  * Topic: `MQTT_TOPIC_ROOT/new`
  * Message (json): 
```
    { 
        "columns": [ "time [s]", "y1", "y2", ... ], 
        "rollover": 200, 
        "args": { "title": "MQTT Plot Demo" }, 
        "layout": "scatter_plot" 
    }
```
  * Declaration of the data column names. The number and names of columns is arbitrary. `rollover` specifies the number of points shown. Once more than that many points are received, the plot scrolls to the left. 
* Json data:
  * Topic: `MQTT_TOPIC_ROOT/add`
  * Message (json):
```
    { 
        "time [s]": 5.39, 
        "y1": float('nan'), 
        "y2": -0.3 
    }
```
  * Data to plot. Typically and (endless) stream.
* Binary data (`from struct import pack`)
  * Topic: `MQTT_TOPIC_ROOT/bin`
  * Message (binary): 
```
        pack("!3f", 3.5, float('nan'), -77)
```
  * Send binary data (32-bit floats), alternative to send json. The number and order of floats sent must match the columns specified during initialization. `bin`, unlike `add`, avoids data allocation on the heap.
* Stop Bokeh server
  * Topic: `MQTT_TOPIC_ROOT/quit`
  * Message: None

First update the configuration file `$IOT_PROJECTS/mqtt_plot/config.py`. Set `MQTT_BROKER` to match the address of your MQTT broker.

Then start the server from the command line:

```bash
cd $IOT_PROJECTS/robot
python -m mqtt_plot
```

Wait for the program to announce it's waiting for data, then launch the data generator (e.g. a microcontroller), first sending to the `new` topic to initialize the plot, followed by data. Open a browser window at the URL indicated by the application (e.g. `http://iot49.local:5006`).

## Testing

To test the application (without setting up a device to send MQTT data), run the code below to send "sample" data (update `MQTT_BROKER` to match your configuration). 

**Note:** the example below starts & stops `mqtt_plot` automatically; do not also start from the command line!

In [28]:
import paho.mqtt.client as mqtt
from struct import pack
from time import sleep
import json, math


# Broker address
MQTT_BROKER = "10.39.40.200"

# stop bokeh server after sending TMAX seconds worth of data
TMAX = 300   

# see config.py
TOPIC_ROOT = "public/vis"


# hack to start mqtt_plot from Python
import subprocess, os
print("starting mqtt_plot ...")
cwd = os.path.expandvars("$IOT_PROJECTS/robot")
proc = subprocess.Popen(["python", "-m", "mqtt_plot"], cwd=cwd, stdout=subprocess.PIPE)
# wait for mqtt_plot to start
proc.stdout.readline()
print(f"connect to bokeh server at http://{os.getenv('DNS_NAME', 'iot49')}.local:5006")


# start MQTT client for sending data
client = mqtt.Client(clean_session=True, protocol=mqtt.MQTTv311)
client.connect(MQTT_BROKER, port=1883, keepalive=60)

print("initializing the plot ...")

client.publish(f"{TOPIC_ROOT}/new", json.dumps({
    "columns": [ "time [s]", "sin", "cos" ],
    "rollover": 200,
    "args": { "title": "MQTT Plot Demo" },
    # "layout": "scatter_plot"
}))

print("sending data ...")

try:
    t = 0
    while t < TMAX:
        binary = True
        if binary:
            client.publish(f"{TOPIC_ROOT}/bin", 
                           pack(f"!3f", 
                                t, 
                                math.sin(t), 
                                math.cos(t) if math.cos(t) > 0 else float('nan')))
        else:
            client.publish(f"{TOPIC_ROOT}/add", 
                           json.dumps({ 
                               "time [s]": t, 
                               "sin": math.sin(t), 
                               "cos": math.cos(t) if math.cos(t) > 0 else float('nan') }))
        t += 0.1
        sleep(0.01)
    # shut down the bokeh server
    print("shutting down the bokeh server")
    client.publish(f"{TOPIC_ROOT}/quit")
except KeyboardInterrupt:
    print("so long")

starting mqtt_plot ...
connect to bokeh server at http://pi4robot.local:5006
initializing the plot ...
sending data ...
shutting down the bokeh server


```{figure} figures/mqtt_plot_demo.png
:width: 500px

MQTT Plot Demo
```

## Customization

MQTT Plot makes use of the [Bokeh Visualization Library](https://bokeh.org/). This library offers extensive control and [customizations](https://docs.bokeh.org/en/latest/docs/gallery.html).

To customize the output of MQTT Plot, edit a copy of the file `mqtt_plot/layouts/line_plot.py`). Then in the `new` topic, specify layout to use for plotting, like so (replace `scatter_plot` with the name of your file, without extension):

```
{ "columns": [ "time [s]", "y1", "y2", ... ], "layout": "scatter_plot" }
```